In [0]:
!pip install transformers accelerate Xformers langchain sentence-transformers einops chromadb unstructured gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [0]:
dbutils.library.restartPython()

In [0]:
import torch
from transformers import pipeline
from transformers import AutoTokenizer

model_name = "databricks/dolly-v2-3b"

tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")

generate_text = pipeline(model=model_name, 
                         torch_dtype=torch.bfloat16,
                         trust_remote_code=True, 
                         device_map="auto", 
                         return_full_text=True,
                         max_new_tokens=256, 
                         top_p=0.95, 
                         top_k=50)

In [0]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline
import torch

hf_pipeline = HuggingFacePipeline(pipeline=generate_text)

# template for an instrution with no input
prompt = PromptTemplate(
    input_variables=["instruction"],
    template="{instruction}")

llm_chain = LLMChain(llm=hf_pipeline, prompt=prompt)

# Test LLM Chain

In [0]:
question = 'Who was Dolly the sheep?'
llm_chain.run(question)

'\nDolly the sheep was a sheep. She was a one-of-a-kind sheep - a mammal that was the first-ever cloned mammal.'

In [0]:
question = 'what are the colors of the rainbow?'
llm_chain.run(question)

'\nThe primary colors are red, orange, yellow, green and blue. Additional colors are purple and indigo.'

In [0]:
# CLEAR CUDA if needed
"""
import torch
import gc
torch.cuda.empty_cache()
gc.collect()
"""

'\nimport torch\nimport gc\ntorch.cuda.empty_cache()\ngc.collect()\n'

# Download HuggingFace Embeddings
Check [MTEB English Leaderboard](https://huggingface.co/spaces/mteb/leaderboard) to make sure you download embeddings with good performance

In [0]:
# Choose one of the top performers from the MTEB English Leaderboard
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings

# top #2 when task = Retrieval June 2023 for under ~500 MB
model_name = "intfloat/e5-base-v2" 

hf = HuggingFaceEmbeddings(model_name=model_name)

No sentence-transformers model found with name /root/.cache/torch/sentence_transformers/intfloat_e5-base-v2. Creating a new one with MEAN pooling.


# Prepare Documents

In [0]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import UnstructuredPDFLoader

# Load the Ontario Building Code using its Sitemap
Ontario Building Code:
https://www.buildingcode.online/

You only need to run the following blocks once, then once saved as 'persistent dir', you can commit to your project and then load, without having to make the vector database each time.

In [0]:
# fixes a bug with asyncio and jupyter
import nest_asyncio

nest_asyncio.apply()     

In [0]:
from langchain.document_loaders.sitemap import SitemapLoader

sitemap_loader = SitemapLoader(web_path="https://www.buildingcode.online/sitemap.xml")
docs = sitemap_loader.load()

Fetching pages: 100%|##########| 2281/2281 [04:13<00:00,  9.01it/s]


In [0]:
# High chunk_overlap to provide more context
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=300)
texts = text_splitter.split_documents(docs)

In [0]:
len(texts)

7465

In [0]:
texts[2]

Document(page_content="Section 1 of the Ontario Building Code\r\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nBuildingCode.Online\n\n\n\nBuilding Codes\n\nHome\nOntario Building Code\nQuebec Building Code\nBritish Columbia Building Code\n\n\nNavigation\n\nSection 1\nSection 3\nSection 4\nSection 5\nSection 6\nSection 7\nSection 8\nSection 9\nSection 10\nSection 11\nSection 12\n\n\n\n\n\n\n\n\n\n\n\n\n\nHome\nSection 1 of the Ontario Building Code\n\n\n\r\nSection 1 of the Ontario Building Code\r\n\t\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nNavigate\n\n\nSection 1\nSection 3\nSection 4\nSection 5\nSection 6\nSection 7\nSection 8\nSection 9\nSection 10\nSection 11\nSection 12\n\n\n\n\n\n\n\n\r\nSection 1 of the Ontario Building Code\r\n\t  \n\n\n1.1.1.1. - Application\n1.1.2.1. - Climatic and Seismic Design Values\n1.1.2.2. - Depth of Frost Penetration\n1.3.1.1. - Effective Date\n1.3.1.2. - Applicable Editions\n1.3.2.1. - Abbreviations of Proper Names\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nThis material 

# Make a Vector Database

In [0]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

## Here is the new embeddings being used
embedding = hf 

vectordb = Chroma.from_documents(documents=texts, 
                                 embedding=embedding,
                                 persist_directory=persist_directory)

vectordb.persist() # Don't forget this!!


In [0]:
# Zip vector database so we can resume later without running the command again
! zip -r db.zip /content/db*

  adding: content/db/ (stored 0%)
  adding: content/db/chroma-collections.parquet (deflated 50%)
  adding: content/db/index/ (stored 0%)
  adding: content/db/index/uuid_to_id_e6002115-cbd5-4752-8643-d2892c6ac2cc.pkl (deflated 41%)
  adding: content/db/index/index_e6002115-cbd5-4752-8643-d2892c6ac2cc.bin (deflated 26%)
  adding: content/db/index/index_metadata_e6002115-cbd5-4752-8643-d2892c6ac2cc.pkl (deflated 14%)
  adding: content/db/index/id_to_uuid_e6002115-cbd5-4752-8643-d2892c6ac2cc.pkl (deflated 37%)
  adding: content/db/chroma-embeddings.parquet (deflated 21%)


# Already made a vector database? Continue here!
If you're following along the original notebook, just load my db.

In [0]:
!git clone https://github.com/socd06/dolly-expert-lite.git

Cloning into 'dolly-expert-lite'...
remote: Enumerating objects: 116, done.
remote: Total 116 (delta 0), reused 0 (delta 0), pack-reused 116
Receiving objects: 100% (116/116), 45.40 MiB | 24.90 MiB/s, done.
Resolving deltas: 100% (57/57), done.


In [0]:
!mv dolly-expert-lite/db db

In [0]:
from langchain.chains import RetrievalQA

from langchain.vectorstores import Chroma

In [0]:
persist_directory = 'db'
vectordb = Chroma(persist_directory=persist_directory,
                   embedding_function=hf)
# vectordb.get()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ <command-588418364077624>:4 in <module>                                                          │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'Chroma' object has no attribute 'get'

Make a basic retriever

In [0]:
retriever = vectordb.as_retriever(search_kwargs={'k':3})

# Test the basic retriever

In [0]:
docs = retriever.get_relevant_documents("What are minimum ceiling height requirements for habitable rooms?")

In [0]:
for doc in docs:
  print(doc)

page_content='9.5.3.1. ceiling Heights of Rooms or Spaces   (1) The ceiling heights of rooms or spaces in residential occupancies and live/work units shall conform to Table 9.5.3.1.  (2) Areas in rooms or spaces over which ceiling height is not less than the minimum specified in Table 9.5.3.1. shall be contiguous with the entry or entries to those rooms or spaces. Table 9.5.3.1.   Room ceiling Heights Forming Part of Sentences 9.5.3.1.(1) and (2)    Item   Column 1 Room or Space   Column 2 Minimum Heights(1)     1.   Living room or space, dining room or space, kitchen or kitchen space   2 300 mm over at least 75% of the required floor area with a clear height of 2 100 mm at any point over the required area     2.   Bedroom or bedroom space   2 300 mm over at least 50% of the required area or 2 100 mm over all of the required floor area.  Any part of the floor having a clear height of less than 1 400 mm shall not be considered in computing the required floor area     3.   basement space

# Make a proper Question Retrieval chain

In [0]:
# Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])      

In [0]:
# from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationBufferWindowMemory

from typing import Dict, Any

# class AnswerConversationBufferMemory(ConversationBufferMemory):
class AnswerConversationBufferMemory(ConversationBufferWindowMemory):
    def save_context(self, inputs: Dict[str, Any], outputs: Dict[str, str]) -> None:
        return super(AnswerConversationBufferMemory, self).save_context(inputs,{'response': outputs['result']})

In [0]:
memory = AnswerConversationBufferMemory(k=3)

qa_chain_with_memory = RetrievalQA.from_chain_type(llm=hf_pipeline, 
                                                   chain_type="stuff", 
                                                   retriever=retriever, 
                                                   return_source_documents=True,
                                                   memory=memory)

In [0]:
# try to set the tone
template = '''
You are the assistant to a tradesperson with knowledge of the Ontario Building Code. You provide specific details using the context given and the users question. 
If you don't know the answer, you truthfully say you don't know and don't try to make up an answer. 
----------------
{context}

Question: {question}
Helpful Answer:'''

In [0]:
qa_chain_with_memory.combine_documents_chain.llm_chain.prompt.template = template
qa_chain_with_memory.combine_documents_chain.llm_chain.prompt.template

"\nYou are the assistant to a tradesperson with knowledge of the Ontario Building Code. You provide specific details using the context given and the users question. \nIf you don't know the answer, you truthfully say you don't know and don't try to make up an answer. \n----------------\n{context}\n\nQuestion: {question}\nHelpful Answer:"

In [0]:
def clean_text(text):
    # Remove excessive whitespace
    cleaned_text = ' '.join(text.split())  
    # Keep max one newline character
    cleaned_text = cleaned_text.replace('\n\n', '\n')  

    return cleaned_text

In [0]:
## Cite sources
def chatbot_llm_response(llm_response):
  text = clean_text(llm_response['result']) + '\nSources:\n'
  for source in llm_response["source_documents"]:
    text += source.metadata['source'] +'\n'
  
  return text

In [0]:
query = "What are the minimum ceiling height requirements for habitable rooms?"
llm_response = qa_chain_with_memory(query)
process_llm_response(llm_response)

In [0]:
query = "provide information on fire safety regulations for commercial"
llm_response = qa_chain_with_memory(query)
processed_response = chatbot_llm_response(llm_response)
print(processed_response)

6. B6 3.1.8.5.(2) (a) Existing functional and sound doors in existing buildings that are either hollow metal or kalamein and containing wired glass at least 6 mm thick and conforming to Sentence 3.1.8.14.(2) are permitted in lieu of doors not required to exceed 45 min, 7. B7 3.1.8.7. to 3.1.8.9. Fire dampers or fire stop flaps are not required to be installed in existing Functional and sound hollow metal or kalamein doors which carry existing 1.5 h labels are acceptable in lieu of current 1.5 h labels and may contain wired glass panels not exceeding 0.0645 m², at least 6 mm thick and conforming to Sentence 3.1.8.14.(2) Reserved for Fire doors, window assemblies or glass blocks used as a closure in a required fire separation shall be installed in conformance with good engineering practice. fire-resistant construction materials including, but not limited to metal, cement, concrete, brick, concrete block, steel, timber and noncombustible construction materials including,
Sources:
https://

In [0]:
query = "What are the accessibility guidelines for barrier-free design in commercial buildings"
llm_response = qa_chain_with_memory(query)
processed_response = chatbot_llm_response(llm_response)
print(processed_response)

This is a good question. The guidelines for commercial buildings are defined in the Ontario Building Code, R.S.O. 1990, Reg. 250, and also available in Book 2, Chapter 2, Section 2.7 and Appendix 2 for guidelines on commercial spaces. The accessible design principles, requirements and guidelines outlined in these documents should assist in the construction of barrier-free commercial buildings. A basic understanding of the principles, requirements and guidelines outlined is required. A comprehensive guide is available on the web. It is recommended to review the web site for details of the barrier-free guidelines by using the acronym FAR (Federal Accessibility Recommendation) and state codes where applicable. For example, where a federal mandate is applicable, the ACM (Access and Coordination Modules). It is also recommended that there is an accessible elevator and that there is a clear path for the wheelchair lift to direct the wheelchair along.
Sources:
https://www.buildingcode.online/

In [0]:
query = "What are the regulations regarding installation of smoke alarms and carbon monoxide detectors in residential buildings?"
llm_response = qa_chain_with_memory(query)
processed_response = chatbot_llm_response(llm_response)
print(processed_response)


The Ontario Building Code (OBC) requires installation of smoke alarms and carbon monoxide alarms in residential buildings. To meet this requirement, the following must be considered: 1. Where required by the OBC, smoke alarms and carbon monoxide alarms must be permanently connected to an electrical circuit and have no disconnect switch between the overcurrent device and the alarm. 2. Where required, smoke alarms and carbon monoxide alarms must be wired such that their activation will activate all alarms within the suite. 3. Where required, smoke alarms and carbon monoxide alarms must be equipped with an alarm that is audible within bedrooms when the intervening doors are closed. 4. Where required, smoke alarms and carbon monoxide alarms must conform to the requirements of Canadian Standards Association (CSA) document 619, "Residential Carbon Monoxide Alarming Devices". 5. Where applicable, smoke alarms and carbon monoxide alarms must conform to the requirements of Underwriters' Laborat

In [0]:
query = "What are the requirements for plumbing venting and drainage systems according to the building code?"
llm_response = qa_chain_with_memory(query)
processed_response = chatbot_llm_response(llm_response)
print(processed_response)

A vent pipethas to be connected to the top of the sump or tank. Every sump or tank that receives sanitary sewage shall be provided with a vent pipethat is connected to the top of the sump or tank.
Sources:
https://www.buildingcode.online/1088.html
https://www.buildingcode.online/1083.html
https://www.buildingcode.online/1099.html



In [0]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.5/471.5 kB 47.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.3/288.3 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.0/137.0 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 9.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 9.3 MB/s eta 0:00:00

In [0]:
import gradio as gr
import random
import time

examples = ["What are the requirements for plumbing venting and drainage systems?",
            "Summarize the electrical code regulation for wiring commercial buildings",
            "Tell me the maximum allowable span for floor joists in residential construction",
            "I'm looking for guidelines for fire assemblies and walls in tall buildings",
            "What are the insulation requirements in new residential constructions?"]

def process_example(args):
    for x in generate(args):
        pass
    return x
    
def generate(instruction): 
    response = qa_chain_with_memory(instruction)    
    processed_response = chatbot_llm_response(response)

    result = ""
    for word in processed_response.split(" "):
        result += word + " "
        yield result


with gr.Blocks(theme=gr.themes.monochrome, analytics_enabled=False) as demo:
    with gr.Column():
        gr.Markdown("""# Dolly-Expert-Lite         
                    Dolly-Expert-Lite is a bot for domain specific question 
                    answering. Currently powered by the new Dolly-v2-3b open 
                    source model. It's expert systems in the era of LLMs!
                    
                    ## Building Code Expert 
                    In this example deployment, Dolly-Expert-Lite retrieves 
                    information via a vector database made using the 
                    [Ontario (Canada) Building Code](https://www.buildingcode.online) 
                    sitemap LangChain loader. For details on the original Dolly 
                    v2 model, please refer to the 
                    [model card](https://huggingface.co/databricks/dolly-v2-12b)
            
                    ### Type in the box below and click to ask the expert!
            
      """
        )

        with gr.Row():
            with gr.Column(scale=3):
                instruction = gr.Textbox(placeholder="Enter your question here", label="Question", elem_id="q-input")

                with gr.Box():
                    gr.Markdown("**Answer**")
                    output = gr.Markdown(elem_id="q-output")
                submit = gr.Button("Generate", variant="primary")            
                clear = gr.Button("Clear", variant="secondary")

                gr.Examples(
                    examples=examples,
                    inputs=[instruction],
                    cache_examples=False,
                    fn=process_example,
                    outputs=[output],
                )        


    submit.click(generate, inputs=[instruction], outputs=[output])
    clear.click(lambda: None, [], [output])
    instruction.submit(generate, inputs=[instruction], outputs=[output])

demo.queue(concurrency_count=16).launch(share=True)

/local_disk0/.ephemeral_nfs/envs/pythonEnv-b7b9c20f-2450-427a-8f01-5af77157c706/lib/python3.10/site-packages/gradio/blocks.py:682: UserWarning: Theme should be a class loaded from gradio.themes
  warnings.warn("Theme should be a class loaded from gradio.themes")


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://07fc88ea416f27e238.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
